In [35]:
import psychofit as psy
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join, splitext
import pandas as pd
import re
from datetime import datetime

In [102]:
def plot_session(subject, date, subsession, exp_ref=None, root_dir=None):
    if rootDir is None:
        rootDir = r'\\zubjects.cortexlab.net\Subjects'
    fullpath = join(rootDir, subject, date, str(subsession))
    load_alfs(fullpath)
    #feedback = np.load(join(fullpath, alfs[0]))
    return
ans = plot_session('ALK081', '2018-08-28', 1)
print(ans)

dict_keys(['cwStimOn', 'cwTrials', 'cwGoCue', 'cwResponse', 'Wheel', 'cwFeedback'])
None


In [36]:
def parse_ref(ref):
    pattern = r'(?P<date>^[0-9\-]+)_(?P<seq>\d+)_(?P<subject>\w+)'
    try:
        out = re.match(pattern,ref).groupdict()
    except:
        raise ValueError('%s could not be parsed', ref)
    return out['subject'], datetime.strptime(out['date'], '%Y-%m-%d'), int(out['seq'])

In [38]:
subject, date, seq = parse_ref('2018-08-27_1_fhuads')
print(type(date))

<class 'datetime.datetime'>


In [40]:
def expPath(ref, rootDir):
    subject, date, seq = parse_ref(ref)
    path = join(rootDir, subject, date.strftime('%Y-%m-%d'), str(seq))
    return path

In [44]:
def is_alf(fileName):
    ''' Return logical array the length of list, of whether a given file name is ALF'''
    pattern = r'(?P<obj>.+)\.(?P<typ>.+)\.(?P<ext>.+)'
    out = re.match(pattern,file_name)
    return out is not None

In [46]:
def alf_parts(fileName):
    ''' Return the object, type and extention for a given ALF file name'''
    '''TODO: Deal with namespaces'''
    obj, typ, ext = file_name.split('.')
    return obj, typ, ext

In [101]:
def load_alfs(fullpath):
    '''Takes a path and loads alfs into dataframes based on object'''
    alfs = [f for f in listdir(fullpath) if (isfile(join(fullpath, f))) & (is_alf(f))]
    parts = [alf_parts(alf) for alf in alfs]
    behaviour = dict.fromkeys(set([x[0] for x in parts]))
    for alf in parts:
        behaviour[alf[0]]
        
    print(behaviour.keys())

In [75]:
print(f[4])
print(is_alf(f[4])[0])

cwFeedback.type.npy
True


In [176]:
def load_behaviour(ref, rootDir=None):
    '''TODO: Deal with namespaces'''
    if rootDir is None:
        rootDir = r'\\zubjects.cortexlab.net\Subjects'
    path = expPath(ref, rootDir)
    alfs = [f for f in listdir(path) if (isfile(join(path, f))) & (is_alf(f))]
    parts = [alf_parts(alf) for alf in alfs]
    # List of 'trials' attributes
    attr = [parts[i][1] for i in range(len(parts)) if parts[i][0].startswith('_ibl_trials')]
    attr.extend(['trialStart', 'trialEnd'])
    # Pull paths of trials ALFs
    trials = [join(path,f) for f in alfs if f.startswith('_ibl_trials')]
    # Load arrays into dictionary
    trialsDict = dict.fromkeys(attr)
    for p,name in zip(trials, attr):
        trialsDict[name] = np.load(p).squeeze()
    # Check all arrays the same length
    lengths = [len(val) for val in [trialsDict.values()]]
    assert len(set(lengths))==1,'Not all arrays in trials the same length'
    # Deal with intervals
    trialsDict['trialStart'] = trialsDict['intervals'][:,0]
    trialsDict['trialEnd'] = trialsDict['intervals'][:,1]
    trialsDict.pop('intervals', None)
    # Create data from from trials dict
    df = pd.DataFrame(trialsDict)
    return df
df = load_behaviour('2018-09-07_1_LEW009')
plot_performance(df)

TypeError: data must be a list or numpy array

In [175]:
def plot_performance(df):
    df['contrast'] = df['contrastRight']-df['contrastLeft']
    contrastSet = df['contrast'].unique()
    choiceSet = np.array(set(df['choice']))
    nn = np.array([sum((df['contrast']==c) & (df['included']==True)) for c in contrastSet])
    pp = np.array([sum((df['contrast']==c) & (df['included']==True) & (df['feedbackType']==1.)) for c in contrastSet])
    if contrastSet.size > 2:
        pars, L = psy.mle_fit_psycho((contrastSet,nn,pp/nn), P_model='erf_psycho')
    else:
        pars = None
    
    return pars